In [1]:
# %%timeit
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, round
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.functions import max, min, sum, mean, when, udf
from pyspark.sql.types import IntegerType, DoubleType
from pyspark.rdd import RDD
from pyspark.sql.dataframe import DataFrame as DataFrame
from pandas.core.frame import DataFrame as PandasDataframe

# Run the date count jobs tasks
from jobs.solana import SparkJob

job = SparkJob()
df = job.create_dataframe()
df.show(10)

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      20000|
|  Python|     100000|
|   Scala|       3000|
+--------+-----------+

